In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import multivariate_normal as mvn

from armored.models import *
from armored.preprocessing import *

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

# set plot parameters
params = {'legend.fontsize': 18,
          'figure.figsize': (16, 12),
          'lines.linewidth': 4,
          'axes.labelsize': 24,
          'axes.titlesize':24,
          'axes.linewidth':5,
          'xtick.labelsize':20,
          'ytick.labelsize':20}
plt.rcParams.update(params)
plt.style.use('seaborn-colorblind')
plt.rcParams['pdf.fonttype'] = 42

np.random.seed(123)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-m6i4zvaw because the default path (/home/jaron/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Set parameters of script

In [2]:
# define k for k-fold (n_splits = k)
n_splits = 20

# number of times to run k-fold x-validation 
# each trial uses different random partitioning of data
n_trials = 10

# import data

In [3]:
df = pd.read_csv("Data/2021_02_19_MultifunctionalDynamicData.csv")

all_experiments = df.Experiments.values
unique_experiments = np.unique(all_experiments)

df.head()

,Experiments,Time,PC_OD,PJ_OD,BV_OD,BF_OD,BO_OD,BT_OD,BC_OD,BY_OD,...,CG_OD,ER_OD,RI_OD,CC_OD,DL_OD,DF_OD,Butyrate,Acetate,Lactate,Succinate
0,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,0,0.000471,0.0,0.000471,0.000000,0.000000,0.000471,0.000471,0.0,...,0.000471,0.000471,0.000471,0.0,0.0,0.000471,0.000000,0.000000,28.000000,0.000000
1,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,16,0.465116,0.0,0.029207,0.000000,0.000000,0.249717,0.500651,0.0,...,0.024339,0.327601,0.001460,0.0,0.0,0.392830,23.092697,47.849302,18.910852,26.141885
2,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,32,0.104523,0.0,0.027928,0.000000,0.000000,0.220107,0.380210,0.0,...,0.020739,0.293384,0.000830,0.0,0.0,0.280111,23.996267,38.915218,17.977137,26.884748
3,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,48,0.124852,0.0,0.012194,0.000000,0.000000,0.268268,0.333970,0.0,...,0.023660,0.188188,0.002366,0.0,0.0,0.224952,24.839219,34.325914,19.406971,31.628061
4,BV-BF-BO-BT-BU-DP-BL-BA-BP-EL-FP-CH-AC-BH-CG-E...,0,0.000000,0.0,0.000388,0.000388,0.000388,0.000388,0.000000,0.0,...,0.000388,0.000388,0.000388,0.0,0.0,0.000000,0.000000,0.000000,28.000000,0.000000


### Define which variables are species and which are metabolites

In [4]:
# specify species and metabolite names 
species = df.columns.values[2:-4]
metabolites =  df.columns.values[-4:]
controls = []
system_variables = np.concatenate((np.array(species), np.array(metabolites)))
system_variables

array(['PC_OD', 'PJ_OD', 'BV_OD', 'BF_OD', 'BO_OD', 'BT_OD', 'BC_OD',
       'BY_OD', 'BU_OD', 'DP_OD', 'BL_OD', 'BA_OD', 'BP_OD', 'CA_OD',
       'EL_OD', 'FP_OD', 'CH_OD', 'AC_OD', 'BH_OD', 'CG_OD', 'ER_OD',
       'RI_OD', 'CC_OD', 'DL_OD', 'DF_OD', 'Butyrate', 'Acetate',
       'Lactate', 'Succinate'], dtype=object)

In [5]:
# dataframe to record likelihoods
LL_df = pd.DataFrame()

In [6]:
# perform k-fold x-validation several times (n_trials) with randomized partitions of data
for k in range(n_trials):
    
    # record log-likelihood of test data using predicted variance
    pred_log_likelihood = []

    # record log-likelihood of test data using fixed variance
    fix_log_likelihood = []
    
    # initialize dataframe to save predictions
    df_kfold = pd.DataFrame()

    # object to get indices of kfold splits
    kf = KFold(n_splits=n_splits, random_state=k, shuffle=True)

    # loop over k-fold splits
    for train_index, test_index in kf.split(unique_experiments):

        # update index values to pull trajectories 
        train_index = np.in1d(all_experiments, unique_experiments[train_index])
        test_index  = np.in1d(all_experiments, unique_experiments[test_index])

        # pull train and test dataframes
        train_df  = df.iloc[train_index].copy() 
        test_df   = df.iloc[test_index].copy() 

        # format data
        train_data, exp_names, N = format_data(train_df, species, metabolites, controls)
        test_data, test_exp_names, _ = format_data(test_df, species, metabolites, controls)

        # transform data
        scaler = ZeroMaxScaler().fit(train_data)
        train_data = scaler.transform(train_data)
        test_data  = scaler.transform(test_data)

        # Apply RNN to predict metabolite concentration on sparsely sampled data:
        brnn = miRNN(n_species=len(species), n_metabolites=len(metabolites), n_controls=0, 
                     n_hidden=16, N=N)
        brnn.fit(train_data)

        # Make predictions
        preds, stdvs, COV = brnn.predict(test_data)
        
        # remove time zero COV
        COV = COV[:, 1:]
        
        ### evaluate log-likelihood using PREDICTED covariance ### 

        # determine shape of model output 
        ns, nt, no = preds.shape

        # subtract a time point to account for initial condition
        nt -= 1

        # compute precision of each prediction 
        Beta = np.linalg.inv(COV)

        # log det of cov
        sum_log_eigs = np.sum(np.log(np.linalg.eigvalsh(COV)))/2.

        # log of exponential of SSE 
        SSE = np.einsum('ntk,ntkl,ntl',test_data[:,1:]-preds[:,1:], Beta, test_data[:,1:]-preds[:,1:])/2.

        # constant term
        const = ns*nt*no/2. * np.log(2*np.pi)

        # compute log likelihood
        pred_log_likelihood.append(-sum_log_eigs -SSE -const)

        ### evaluate log-likelihood using FIXED covariance ### 

        fixed_COV  = brnn.BetaInv
        fixed_Beta = np.linalg.inv(fixed_COV)

        # log det of cov
        sum_log_eigs = ns*nt*np.sum(np.log(np.linalg.eigvalsh(fixed_COV)))/2.

        # log of exponential of SSE 
        SSE = np.einsum('ntk,kl,ntl',test_data[:,1:]-preds[:,1:], fixed_Beta, test_data[:,1:]-preds[:,1:])/2.

        # constant term
        const = ns*nt*no/2. * np.log(2*np.pi)

        # compute log likelihood
        fix_log_likelihood.append(-sum_log_eigs -SSE -const)
        
        # Transform values back to original scale
        preds = scaler.inverse_transform(preds)
        stdvs = scaler.inverse_transform_stdv(stdvs)
        COV   = scaler.inverse_transform_cov(COV)
        test_data = scaler.inverse_transform(test_data)

        # save data
        pred_df_kfold = pd.DataFrame()
        for i, exp_name in enumerate(test_exp_names):
            pred_df = pd.DataFrame()
            pred_df['Experiments'] = [exp_name]*4
            pred_df['Time'] = [0, 16, 32, 48]
            for j, variable in enumerate(system_variables):
                pred_df[variable + " true"] = test_data[i, :, j]
                pred_df[variable + " pred"] = preds[i, :, j]
                pred_df[variable + " stdv"] = stdvs[i, :, j]
            pred_df_kfold = pd.concat((pred_df_kfold, pred_df))
        df_kfold = pd.concat((df_kfold, pred_df_kfold))
        
    # save likelihood results
    LL_df_k = pd.DataFrame()
    LL_df_k['k'] = [k + 1]
    LL_df_k['LL Pred'] = [np.sum(pred_log_likelihood)]
    LL_df_k['LL Fix']  = [np.sum(fix_log_likelihood)]
    LL_df = pd.concat((LL_df, LL_df_k))
    LL_df.to_csv(f"Fig2/MiRNN_LL_results.csv", index=False)
        
    # save kfold predictions
    df_kfold.to_csv(f"Fig2/Kfold/MiRNN_{n_splits}_fold_{k+1}.csv", index=False)

Epoch: 0, Loss: 732.00322, Residuals: -8.03454, Convergence:   inf
Epoch: 10, Loss: 669.66174, Residuals: -7.40201, Convergence: 0.010511
Epoch: 20, Loss: 597.64044, Residuals: -6.73303, Convergence: 0.011854
Epoch: 30, Loss: 530.90704, Residuals: -6.09958, Convergence: 0.011900
Epoch: 40, Loss: 472.59583, Residuals: -5.51444, Convergence: 0.011516
Epoch: 50, Loss: 422.59552, Residuals: -4.98208, Convergence: 0.011009
Epoch: 60, Loss: 380.15326, Residuals: -4.50042, Convergence: 0.010331
Epoch: 70, Loss: 344.50797, Residuals: -4.06688, Convergence: 0.009573
Epoch: 80, Loss: 314.47980, Residuals: -3.67414, Convergence: 0.008825
Epoch: 90, Loss: 289.67126, Residuals: -3.32541, Convergence: 0.008041
Epoch: 100, Loss: 268.31824, Residuals: -3.00771, Convergence: 0.007272
Epoch: 110, Loss: 250.36038, Residuals: -2.72005, Convergence: 0.006634
Epoch: 120, Loss: 235.27425, Residuals: -2.46031, Convergence: 0.005938
Epoch: 130, Loss: 222.62404, Residuals: -2.22751, Convergence: 0.005251
Epoch:


Epoch: 279, Evidence: 3846.06128, Convergence: 0.692389
Epoch: 280, Loss: 1070.39966, Residuals: -0.50865, Convergence: 0.001514
Epoch: 290, Loss: 1055.70422, Residuals: -0.46119, Convergence: 0.001286
Epoch: 300, Loss: 1043.38147, Residuals: -0.41742, Convergence: 0.001103
Epoch: 310, Loss: 1032.35083, Residuals: -0.37768, Convergence: 0.001000

Epoch: 311, Evidence: 5128.01367, Convergence: 0.249990
Epoch: 320, Loss: 1512.15967, Residuals: -0.34134, Convergence: 0.001062

Epoch: 324, Evidence: 5598.98096, Convergence: 0.084117
Epoch: 330, Loss: 1759.77417, Residuals: -0.30833, Convergence: 0.001032

Epoch: 333, Evidence: 5750.76660, Convergence: 0.026394

Epoch: 336, Evidence: 5802.95801, Convergence: 0.008994
Epoch: 340, Loss: 1918.45605, Residuals: -0.27873, Convergence: 0.001023

Epoch: 341, Evidence: 5831.63770, Convergence: 0.004918

Epoch: 344, Evidence: 5846.08594, Convergence: 0.002471

Epoch: 346, Evidence: 5855.17383, Convergence: 0.001552

Epoch: 347, Evidence: 5859.63770


Epoch: 431, Evidence: 6026.83301, Convergence: 0.000927

Epoch: 435, Evidence: 6034.69336, Convergence: 0.002228
Epoch: 440, Loss: 1941.33911, Residuals: 0.00210, Convergence: 0.001216

Epoch: 441, Evidence: 6045.81445, Convergence: 0.001839

Epoch: 446, Evidence: 6056.47607, Convergence: 0.001760
Epoch: 450, Loss: 1940.75793, Residuals: 0.01093, Convergence: 0.000904

Epoch: 450, Evidence: 6064.12500, Convergence: 0.001261
Epoch: 450, Loss: 1949.46033, Residuals: 0.01093, Convergence:   inf

Epoch: 454, Evidence: 6071.49121, Convergence: 0.001213

Epoch: 457, Evidence: 6085.84375, Convergence: 0.002358
Epoch: 460, Loss: 1940.16650, Residuals: 0.02246, Convergence: 0.001009

Epoch: 461, Evidence: 6091.94531, Convergence: 0.001002

Epoch: 463, Evidence: 6096.30811, Convergence: 0.000716

Epoch: 467, Evidence: 6096.19580, Convergence: 0.000697
Epoch: 470, Loss: 1940.68652, Residuals: 0.01730, Convergence: -0.005206
Epoch: 470, Loss: 1941.12061, Residuals: 0.01453, Convergence: -0.005428

Epoch: 450, Loss: 1945.70300, Residuals: 0.00634, Convergence: 0.001137

Epoch: 452, Evidence: 6002.12988, Convergence: 0.001289

Epoch: 455, Evidence: 6007.96143, Convergence: 0.000971

Epoch: 457, Evidence: 6012.41162, Convergence: 0.001710
Epoch: 460, Loss: 1937.60571, Residuals: 0.01303, Convergence: 0.002599

Epoch: 465, Evidence: 6028.72754, Convergence: 0.002706
Epoch: 470, Loss: 1927.78894, Residuals: 0.01715, Convergence: 0.001091

Epoch: 471, Evidence: 6043.93848, Convergence: 0.002517

Epoch: 474, Evidence: 6054.48145, Convergence: 0.001741

Epoch: 478, Evidence: 6062.51367, Convergence: 0.001325
Epoch: 480, Loss: 1936.99902, Residuals: 0.01593, Convergence: 0.001103

Epoch: 481, Evidence: 6068.23389, Convergence: 0.000943

Epoch: 485, Evidence: 6072.59570, Convergence: 0.001660
Epoch: 490, Loss: 1925.40625, Residuals: 0.01583, Convergence: 0.000307

Epoch: 490, Evidence: 6080.18262, Convergence: 0.001248
Epoch: 490, Loss: 1933.88940, Residuals: 0.01583, Convergence:   inf



Epoch: 120, Loss: 236.20143, Residuals: -2.42513, Convergence: 0.005869
Epoch: 130, Loss: 223.98126, Residuals: -2.19960, Convergence: 0.005254
Epoch: 140, Loss: 213.34900, Residuals: -1.99258, Convergence: 0.004675
Epoch: 150, Loss: 204.38869, Residuals: -1.80380, Convergence: 0.004024
Epoch: 160, Loss: 196.84784, Residuals: -1.63350, Convergence: 0.003465
Epoch: 170, Loss: 190.46263, Residuals: -1.47890, Convergence: 0.003060
Epoch: 180, Loss: 185.11725, Residuals: -1.33983, Convergence: 0.002533

Epoch: 189, Evidence: 1207.20923, Convergence:   inf
Epoch: 190, Loss: 577.68481, Residuals: -1.24442, Convergence: -0.002988
Epoch: 190, Loss: 577.93164, Residuals: -1.24597, Convergence: -0.003414
Epoch: 190, Loss: 575.83411, Residuals: -1.24056, Convergence: 0.000216

Epoch: 190, Evidence: 3772.65039, Convergence: 0.680010
Epoch: 190, Loss: 1173.64062, Residuals: -1.24056, Convergence:   inf

Epoch: 191, Evidence: 4814.30859, Convergence: 0.216367

Epoch: 192, Evidence: 5157.95020, Conve

...


Epoch: 277, Evidence: 5596.93652, Convergence: 0.000259

Epoch: 278, Evidence: 5597.96143, Convergence: 0.000442

Epoch: 279, Evidence: 5598.78369, Convergence: 0.000589
Epoch: 0, Loss: 743.15886, Residuals: -7.96896, Convergence:   inf
Epoch: 10, Loss: 679.73145, Residuals: -7.34185, Convergence: 0.010557
Epoch: 20, Loss: 606.25989, Residuals: -6.67641, Convergence: 0.011900
Epoch: 30, Loss: 537.91968, Residuals: -6.04560, Convergence: 0.011995
Epoch: 40, Loss: 477.97647, Residuals: -5.46453, Convergence: 0.011703
Epoch: 50, Loss: 426.71515, Residuals: -4.93704, Convergence: 0.011153
Epoch: 60, Loss: 383.34125, Residuals: -4.46005, Convergence: 0.010476
Epoch: 70, Loss: 346.86951, Residuals: -4.02969, Convergence: 0.009693
Epoch: 80, Loss: 316.64032, Residuals: -3.64571, Convergence: 0.008921
Epoch: 90, Loss: 290.90857, Residuals: -3.29594, Convergence: 0.008162
Epoch: 100, Loss: 269.34317, Residuals: -2.98047, Convergence: 0.007407
Epoch: 110, Loss: 251.28391, Residuals: -2.69561, C


Epoch: 470, Evidence: 6090.20752, Convergence: 0.000971
Epoch: 470, Loss: 1953.52002, Residuals: -0.00463, Convergence:   inf

Epoch: 471, Evidence: 6090.94189, Convergence: 0.001091

Epoch: 472, Evidence: 6091.39160, Convergence: 0.000074

Epoch: 473, Evidence: 6091.65820, Convergence: 0.000118

Epoch: 474, Evidence: 6091.85547, Convergence: 0.000150
Epoch: 0, Loss: 756.38585, Residuals: -8.33848, Convergence:   inf
Epoch: 10, Loss: 690.34540, Residuals: -7.67698, Convergence: 0.010740
Epoch: 20, Loss: 615.17114, Residuals: -6.98168, Convergence: 0.011989
Epoch: 30, Loss: 545.53436, Residuals: -6.32253, Convergence: 0.012131
Epoch: 40, Loss: 484.33997, Residuals: -5.71548, Convergence: 0.011814
Epoch: 50, Loss: 431.93753, Residuals: -5.16410, Convergence: 0.011245
Epoch: 60, Loss: 387.60602, Residuals: -4.66538, Convergence: 0.010578
Epoch: 70, Loss: 350.35910, Residuals: -4.21471, Convergence: 0.009809
Epoch: 80, Loss: 319.26965, Residuals: -3.80779, Convergence: 0.008927
Epoch: 90,

Epoch: 170, Loss: 193.36925, Residuals: -1.48786, Convergence: 0.003012

Epoch: 178, Evidence: 1291.24426, Convergence:   inf
Epoch: 180, Loss: 600.58179, Residuals: -1.35341, Convergence: 0.003650
Epoch: 190, Loss: 580.76556, Residuals: -1.22570, Convergence: 0.003124
Epoch: 200, Loss: 564.28339, Residuals: -1.10997, Convergence: 0.002693
Epoch: 210, Loss: 550.18365, Residuals: -1.00769, Convergence: 0.002299
Epoch: 220, Loss: 538.58496, Residuals: -0.91279, Convergence: 0.001995
Epoch: 230, Loss: 528.79144, Residuals: -0.82697, Convergence: 0.001725
Epoch: 240, Loss: 520.52954, Residuals: -0.74978, Convergence: 0.001505
Epoch: 250, Loss: 513.28973, Residuals: -0.67924, Convergence: 0.001275

Epoch: 259, Evidence: 3880.79785, Convergence: 0.667274
Epoch: 260, Loss: 1111.45691, Residuals: -0.62631, Convergence: 0.000204

Epoch: 260, Evidence: 5132.85352, Convergence: 0.243930
Epoch: 260, Loss: 1586.34961, Residuals: -0.62631, Convergence:   inf

Epoch: 261, Evidence: 5541.23730, Conver

Epoch: 10, Loss: 684.62018, Residuals: -7.35754, Convergence: 0.010506
Epoch: 20, Loss: 611.03754, Residuals: -6.69050, Convergence: 0.011879
Epoch: 30, Loss: 542.20642, Residuals: -6.05798, Convergence: 0.012029
Epoch: 40, Loss: 481.75885, Residuals: -5.47576, Convergence: 0.011726
Epoch: 50, Loss: 429.99957, Residuals: -4.94729, Convergence: 0.011177
Epoch: 60, Loss: 386.12561, Residuals: -4.46920, Convergence: 0.010505
Epoch: 70, Loss: 349.27747, Residuals: -4.03737, Convergence: 0.009760
Epoch: 80, Loss: 318.28159, Residuals: -3.64822, Convergence: 0.008979
Epoch: 90, Loss: 292.76392, Residuals: -3.30199, Convergence: 0.008156
Epoch: 100, Loss: 271.06894, Residuals: -2.98588, Convergence: 0.007383
Epoch: 110, Loss: 252.85545, Residuals: -2.70132, Convergence: 0.006603
Epoch: 120, Loss: 237.67793, Residuals: -2.44625, Convergence: 0.005551
Epoch: 130, Loss: 224.80414, Residuals: -2.21411, Convergence: 0.005261
Epoch: 140, Loss: 213.98718, Residuals: -2.00633, Convergence: 0.004732
E

Epoch: 190, Loss: 189.16692, Residuals: -1.25818, Convergence: 0.002356
Epoch: 200, Loss: 185.17178, Residuals: -1.14274, Convergence: 0.002018

Epoch: 206, Evidence: 1178.29834, Convergence:   inf
Epoch: 210, Loss: 569.34467, Residuals: -1.03674, Convergence: 0.002671
Epoch: 220, Loss: 555.53619, Residuals: -0.94057, Convergence: 0.002293
Epoch: 230, Loss: 543.85895, Residuals: -0.85187, Convergence: 0.002004
Epoch: 240, Loss: 534.00055, Residuals: -0.77323, Convergence: 0.001724
Epoch: 250, Loss: 525.61957, Residuals: -0.70119, Convergence: 0.001497
Epoch: 260, Loss: 518.38831, Residuals: -0.63531, Convergence: 0.001302
Epoch: 270, Loss: 512.14673, Residuals: -0.57655, Convergence: 0.001143
Epoch: 280, Loss: 506.74521, Residuals: -0.52291, Convergence: 0.001004

Epoch: 281, Evidence: 3767.33301, Convergence: 0.687233
Epoch: 290, Loss: 1091.84668, Residuals: -0.47383, Convergence: 0.001191
Epoch: 300, Loss: 1080.00159, Residuals: -0.43036, Convergence: 0.001031

Epoch: 303, Evidence: 